In [ ]:
import numpy
import batman
import matplotlib.pyplot as plt
from matplotlib import rcParams; rcParams["figure.dpi"] = 150

numpy.random.seed(seed=0)  # reproducibility

# Create test data
time_start = 3.14
data_duration = 40
samples_per_day = 48
samples = int(data_duration * samples_per_day)
time = numpy.linspace(time_start, time_start + data_duration, samples)

# Use batman to create transits
ma = batman.TransitParams()
ma.t0 = time_start  # time of inferior conjunction; first transit is X days after start
ma.per = 10.123  # orbital period
ma.rp = 6371 / 696342  # 6371 planet radius (in units of stellar radii)
ma.a = 19  # semi-major axis (in units of stellar radii)
ma.inc = 90  # orbital inclination (in degrees)
ma.ecc = 0  # eccentricity
ma.w = 90  # longitude of periastron (in degrees)
ma.u = [0.4, 0.4]  # limb darkening coefficients
ma.limb_dark = "quadratic"  # limb darkening model
m = batman.TransitModel(ma, time)  # initializes model
synthetic_signal = m.light_curve(ma)  # calculates light curve

# Create noise and merge with flux
ppm = 50  # Noise level in parts per million
noise = numpy.random.normal(0, 10**-6 * ppm, int(samples))
flux = synthetic_signal + noise

# Plot raw data
plt.figure()
ax = plt.gca()
ax.scatter(time, flux, color='black', s=1)
ax.set_ylabel("Flux")
ax.set_xlabel("Time (days)")
plt.xlim(min(time), max(time))
plt.ylim(0.999, 1.001);

In [ ]:
from transitleastsquares import transitleastsquares
model = transitleastsquares(time, flux)
results = model.power()

In [ ]:
print('Period', format(results.period, '.5f'), 'd')
print(len(results.transit_times), 'transit times in time series:', \
        ['{0:0.5f}'.format(i) for i in results.transit_times])
print('Transit depth', format(results.depth, '.5f'))
print('Best duration (days)', format(results.duration, '.5f'))
print('Signal detection efficiency (SDE):', results.SDE)

In [ ]:
plt.figure()
ax = plt.gca()
ax.axvline(results.period, alpha=0.4, lw=3)
plt.xlim(numpy.min(results.periods), numpy.max(results.periods))
for n in range(2, 10):
    ax.axvline(n*results.period, alpha=0.4, lw=1, linestyle="dashed")
    ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed")
plt.ylabel(r'SDE')
plt.xlabel('Period (days)')
plt.plot(results.periods, results.power, color='black', lw=0.5)
plt.xlim(0, max(results.periods))

plt.figure()
plt.plot(results.model_folded_phase, results.model_folded_model, color='red')
plt.scatter(results.folded_phase, results.folded_y, color='blue', s=10, alpha=0.5, zorder=2)
plt.xlim(0.48, 0.52)
plt.ticklabel_format(useOffset=False)
plt.xlabel('Phase')
plt.ylabel('Relative flux');

In [ ]:
from scipy import stats
bins = 500
bin_means, bin_edges, binnumber = stats.binned_statistic(
    results.folded_phase,
    results.folded_y,
    statistic='mean',
    bins=bins)
bin_stds, _, _ = stats.binned_statistic(
    results.folded_phase,
    results.folded_y,
    statistic='std',
    bins=bins)
bin_width = (bin_edges[1] - bin_edges[0])
bin_centers = bin_edges[1:] - bin_width/2

plt.plot(results.model_folded_phase, results.model_folded_model, color='red')
plt.scatter(results.folded_phase, results.folded_y, color='blue', s=10, alpha=0.5, zorder=2)
plt.errorbar(
    bin_centers,
    bin_means,
    yerr=bin_stds/2,
    xerr=bin_width/2,
    marker='o',
    markersize=8,
    color='black',
    #capsize=10,
    linestyle='none')
plt.xlim(0.48, 0.52)
plt.ticklabel_format(useOffset=False)
plt.xlabel('Phase')
plt.ylabel('Relative flux');

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import eleanor
import matplotlib.pyplot as plt
import numpy as np

star = eleanor.Source(tic=91961, sector=11)
data = eleanor.TargetData(star, height=15, width=15, bkg_size=31, do_psf=False, do_pca=True, regressors='corner')
plt.figure(figsize=(15,5))
q = data.quality == 0

lk = data.to_lightkurve()
period = np.linspace(1, 15, 10000)
bls = lk.to_periodogram(method='bls', period=period, frequency_factor=500)

period = bls.period_at_max_power.value * 2
t0 = bls.transit_time_at_max_power.value
duration = bls.duration_at_max_power

period, t0, duration